In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-2.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-2        9    6    5    0   13   16     9   1  12  ...  0.000000   
1      prop-2        9    4    4    0   14   20     4   1  13  ...  0.400000   
2      prop-2        9    6    4    0   28   41    15   1  27  ...  0.000000   
3      prop-2        9    2    3    0   16   13     1   1  15  ...  0.000000   
4      prop-2        9    2    3    0   17   19     1   1  16  ...  0.000000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
23009  prop-2      192    2    3    0   16   24     1   0  16  ...  0.000000   
23010  prop-2      192    4    5    0    9   13     4   0   9  ...  0.000000   
23011  prop-2      192   11    1    0   41  113    27   1  40  ...  0.909091   
23012  prop-2      192    2    3    0    9   11     1   0   9  ...  1.000000   
23013  prop-2      192    6    6    0   17   22     3   0  17  ...  0.000000   

       moa       mfa       cam  ic  cbm         amc  max_cc   avg_cc  bug  
0        0  0.846154  0.440000   2    4    8.000000       1   0.6667    0  
1        0  0.959184  0.555556   3    5   18.250000       1   0.5000    0  
2        0  0.857143  0.404762   1    3   26.333333       5   1.5000    0  
3        0  0.875000  0.625000   1    1   40.000000       7   3.5000    0  
4        0  0.875000  0.625000   1    1   47.000000       1   0.5000    0  
...    ...       ...       ...  ..  ...         ...     ...      ...  ...  
23009    0  0.875000  0.625000   1    1  109.000000      27  13.5000    0  
23010    0  0.913043  0.555556   2    2    9.000000       1   0.5000    0  
23011    1  0.000000  0.363636   0    0  212.272727      15   3.5455    2  
23012    0  0.875000  0.625000   1    1   30.000000       5   2.5000    0  
23013    0  0.840000  0.733333   2    3   12.666667       1   0.6667    0  

[23014 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 9]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0     prop-2        9    6    5    0   13   16     9   1  12  ...  0.000000   
1     prop-2        9    4    4    0   14   20     4   1  13  ...  0.400000   
2     prop-2        9    6    4    0   28   41    15   1  27  ...  0.000000   
3     prop-2        9    2    3    0   16   13     1   1  15  ...  0.000000   
4     prop-2        9    2    3    0   17   19     1   1  16  ...  0.000000   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
4250  prop-2        9    4    7    0    9   13     4   0   9  ...  0.000000   
4251  prop-2        9    2    3    0   15   15     1   1  14  ...  0.000000   
4252  prop-2        9    5    6    0    3    9    10   1   2  ...  0.000000   
4253  prop-2        9    4    4    0   10   21     4   0  10  ...  0.333333   
4254  prop-2        9    2    2    0   11   14     1   1  10  ...  0.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0       0  0.846154  0.440000   2    4   8.000000       1  0.6667    0  
1       0  0.959184  0.555556   3    5  18.250000       1  0.5000    0  
2       0  0.857143  0.404762   1    3  26.333333       5  1.5000    0  
3       0  0.875000  0.625000   1    1  40.000000       7  3.5000    0  
4       0  0.875000  0.625000   1    1  47.000000       1  0.5000    0  
...   ...       ...       ...  ..  ...        ...     ...     ...  ...  
4250    0  0.923077  0.555556   2    2   9.000000       1  0.5000    0  
4251    0  0.875000  0.625000   2    2  27.000000       5  2.5000    0  
4252    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
4253    0  0.953488  0.666667   1    1  22.250000       1  0.5000    0  
4254    0  0.800000  0.666667   1    1  25.500000       1  0.5000    0  

[4255 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3   loc       dam  \
2205    3    3    0   17   24     3   4  13    2  2.000000   116  0.000000   
2619   24    3    5   80  230   266   5  75    2  1.013043  2125  0.333333   
3419    2    6   11   15    4     1  11   4    2  2.000000     8  0.000000   
2631   11    3    0   20   50     0   0  20    5  0.758333   776  0.750000   
952     2    3    0   22   35     1   2  20    2  1.000000   158  0.000000   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   ...       ...   
476     1    1    0    1    3     0   0   1    0  2.000000    19  0.000000   
2643    2    2    0    3    3     1   1   2    2  2.000000     8  0.000000   
764     9    1    0    1   10    28   1   0    9  0.925000    45  1.000000   
1483    5    5    0    9   14     8   1   8    1  1.000000    45  0.000000   
2512    2    1    0   25   27     1   0  25    0  1.000000   131  1.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
2205    0  0.777778  0.666667   1    1  37.666667       1  0.6667  
2619    0  0.241379  0.251553   1    1  86.291667      18  2.9583  
3419    0  0.958333  0.666667   1    1   3.000000       1  0.5000  
2631    0  0.666667  0.366667   0    0  68.454545       7  3.6364  
952     0  0.875000  0.625000   1    1  76.500000       5  2.5000  
...   ...       ...       ...  ..  ...        ...     ...     ...  
476     0  0.000000  0.000000   0    0  11.000000       0  0.0000  
2643    0  0.937500  0.666667   0    0   3.000000       1  0.5000  
764     0  0.000000  0.481481   0    0   3.444444       1  0.8889  
1483    0  0.880000  0.500000   2    5   7.400000       1  0.6000  
2512    0  0.000000  1.000000   0    0  52.000000       0  0.0000  

[3404 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
59      6    6    0   23   19     3  10  13    1  0.866667   72  0.000000   
445     1    1    0    1    3     0   0   1    0  2.000000   22  0.000000   
1636    2    4    0    1    3     1   0   1    1  2.000000    7  0.000000   
1142    5    6    0   14   17     4   1  13    1  0.916667   60  0.000000   
1718    2    3    0   15   21     1   1  14    2  2.000000   85  0.000000   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
443     1    1    0    1    3     0   0   1    0  2.000000   16  0.000000   
743     5    6    0   27   23     4   8  19    1  0.916667   92  0.000000   
1344    4    7    0   10   13     4   2   8    1  1.000000   43  0.000000   
3073    4    1    0   13   32     0   1  12    2  1.027778  190  0.833333   
1131    7    5    0   27   30    15   1  26    1  0.944444  134  0.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
59      0  0.857143  0.733333   2    3  10.500000       1  0.6667  
445     0  0.000000  0.000000   0    0  11.000000       0  0.0000  
1636    0  0.875000  1.000000   1    2   2.500000       1  0.5000  
1142    0  0.884615  0.666667   2    3  10.400000       1  0.6000  
1718    0  0.875000  0.625000   1    1  41.500000       1  0.5000  
...   ...       ...       ...  ..  ...        ...     ...     ...  
443     0  0.000000  0.000000   0    0  11.000000       0  0.0000  
743     0  0.884615  0.666667   2    3  16.800000       1  0.6000  
1344    0  0.923077  0.555556   2    2   9.000000       1  0.5000  
3073    0  0.000000  0.500000   0    0  43.500000       2  1.2500  
1131    0  0.807692  0.611111   2    5  17.714286       1  0.7143  

[851 rows x 20 columns]

In [9]:
y_train = train['bug']
y_test = test['bug']

In [10]:
y_train

2205    0
2619    0
3419    0
2631    1
952     0
       ..
476     0
2643    0
764     0
1483    2
2512    0
Name: bug, Length: 3404, dtype: int64

In [11]:
y_test

59      0
445     0
1636    0
1142    0
1718    0
       ..
443     0
743     0
1344    0
3073    0
1131    0
Name: bug, Length: 851, dtype: int64

In [12]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [13]:
x_train_np

array([[ 3.        ,  3.        ,  0.        , ..., 37.66666667,
         1.        ,  0.6667    ],
       [24.        ,  3.        ,  5.        , ..., 86.29166667,
        18.        ,  2.9583    ],
       [ 2.        ,  6.        , 11.        , ...,  3.        ,
         1.        ,  0.5       ],
       ...,
       [ 9.        ,  1.        ,  0.        , ...,  3.44444444,
         1.        ,  0.8889    ],
       [ 5.        ,  5.        ,  0.        , ...,  7.4       ,
         1.        ,  0.6       ],
       [ 2.        ,  1.        ,  0.        , ..., 52.        ,
         0.        ,  0.        ]])

In [14]:
x_test_np

array([[ 6.        ,  6.        ,  0.        , ..., 10.5       ,
         1.        ,  0.6667    ],
       [ 1.        ,  1.        ,  0.        , ..., 11.        ,
         0.        ,  0.        ],
       [ 2.        ,  4.        ,  0.        , ...,  2.5       ,
         1.        ,  0.5       ],
       ...,
       [ 4.        ,  7.        ,  0.        , ...,  9.        ,
         1.        ,  0.5       ],
       [ 4.        ,  1.        ,  0.        , ..., 43.5       ,
         2.        ,  1.25      ],
       [ 7.        ,  5.        ,  0.        , ..., 17.71428571,
         1.        ,  0.7143    ]])

In [15]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [16]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [17]:
train_x

array([[[ 3.        ,  3.        ,  0.        , ..., 37.66666667,
          1.        ,  0.6667    ]],

       [[24.        ,  3.        ,  5.        , ..., 86.29166667,
         18.        ,  2.9583    ]],

       [[ 2.        ,  6.        , 11.        , ...,  3.        ,
          1.        ,  0.5       ]],

       ...,

       [[ 9.        ,  1.        ,  0.        , ...,  3.44444444,
          1.        ,  0.8889    ]],

       [[ 5.        ,  5.        ,  0.        , ...,  7.4       ,
          1.        ,  0.6       ]],

       [[ 2.        ,  1.        ,  0.        , ..., 52.        ,
          0.        ,  0.        ]]])

In [18]:
test_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
def define_model():
    # Designing and initializing the model.
    model = Sequential()
    model.add(LSTM(200, return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(50, dropout = 0.2, return_sequences=True))
    model.add(LSTM(20, return_sequences=False))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )
    return model

In [20]:
y = list(test_y)

In [21]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(10):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/propv9/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(test_x, test_y)
    total_scores.append(score) 
    predictions = model.predict(test_x)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
35/35 [==============================] - 8s 29ms/step - loss: 0.1486 - mse: 0.1486 - mae: 0.0975 - root_mean_squared_error: 0.3855 - mean_squared_logarithmic_error: 0.0346
Epoch 2/100
35/35 [==============================] - 1s 23ms/step - loss: 0.1460 - mse: 0.1460 - mae: 0.1106 - root_mean_squared_error: 0.3821 - mean_squared_logarithmic_error: 0.0341
Epoch 3/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1429 - mse: 0.1429 - mae: 0.1032 - root_mean_squared_error: 0.3780 - mean_squared_logarithmic_error: 0.0332
Epoch 4/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1411 - mse: 0.1411 - mae: 0.0948 - root_mean_squared_error: 0.3756 - mean_squared_logarithmic_error: 0.0326
Epoch 5/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1406 - mse: 0.1406 - mae: 0.1012 - root_mean_squared_error: 0.3750 - mean_squared_logarithmic_error: 0.0337
Epoch 6/100
35/35 [==============================] - 1s 22ms/step - lo

35/35 [==============================] - 1s 22ms/step - loss: 0.1223 - mse: 0.1223 - mae: 0.1035 - root_mean_squared_error: 0.3497 - mean_squared_logarithmic_error: 0.0309
Epoch 89/100
35/35 [==============================] - 1s 20ms/step - loss: 0.1108 - mse: 0.1108 - mae: 0.0925 - root_mean_squared_error: 0.3328 - mean_squared_logarithmic_error: 0.0292
Epoch 90/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1144 - mse: 0.1144 - mae: 0.0900 - root_mean_squared_error: 0.3383 - mean_squared_logarithmic_error: 0.0293
Epoch 91/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1120 - mse: 0.1120 - mae: 0.0940 - root_mean_squared_error: 0.3346 - mean_squared_logarithmic_error: 0.0291
Epoch 92/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1104 - mse: 0.1104 - mae: 0.0903 - root_mean_squared_error: 0.3323 - mean_squared_logarithmic_error: 0.0287
Epoch 93/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1157 - mse:

35/35 [==============================] - 1s 21ms/step - loss: 0.0955 - mse: 0.0955 - mae: 0.0752 - root_mean_squared_error: 0.3090 - mean_squared_logarithmic_error: 0.0258
Epoch 74/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1034 - mse: 0.1034 - mae: 0.0802 - root_mean_squared_error: 0.3215 - mean_squared_logarithmic_error: 0.0266
Epoch 75/100
35/35 [==============================] - 1s 22ms/step - loss: 0.0938 - mse: 0.0938 - mae: 0.0788 - root_mean_squared_error: 0.3062 - mean_squared_logarithmic_error: 0.0262
Epoch 76/100
35/35 [==============================] - 1s 21ms/step - loss: 0.0977 - mse: 0.0977 - mae: 0.0768 - root_mean_squared_error: 0.3125 - mean_squared_logarithmic_error: 0.0263
Epoch 77/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1223 - mse: 0.1223 - mae: 0.0801 - root_mean_squared_error: 0.3497 - mean_squared_logarithmic_error: 0.0321
Epoch 78/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1287 - mse:

35/35 [==============================] - 1s 21ms/step - loss: 0.1120 - mse: 0.1120 - mae: 0.0901 - root_mean_squared_error: 0.3347 - mean_squared_logarithmic_error: 0.0287
Epoch 59/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1120 - mse: 0.1120 - mae: 0.0926 - root_mean_squared_error: 0.3346 - mean_squared_logarithmic_error: 0.0283
Epoch 60/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1057 - mse: 0.1057 - mae: 0.0904 - root_mean_squared_error: 0.3251 - mean_squared_logarithmic_error: 0.0277
Epoch 61/100
35/35 [==============================] - 1s 20ms/step - loss: 0.1061 - mse: 0.1061 - mae: 0.0914 - root_mean_squared_error: 0.3258 - mean_squared_logarithmic_error: 0.0281
Epoch 62/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1076 - mse: 0.1076 - mae: 0.0895 - root_mean_squared_error: 0.3281 - mean_squared_logarithmic_error: 0.0272
Epoch 63/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1049 - mse:

Epoch 1/100
35/35 [==============================] - 8s 35ms/step - loss: 0.1519 - mse: 0.1519 - mae: 0.0608 - root_mean_squared_error: 0.3897 - mean_squared_logarithmic_error: 0.0354
Epoch 2/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1519 - mse: 0.1519 - mae: 0.0608 - root_mean_squared_error: 0.3897 - mean_squared_logarithmic_error: 0.0354
Epoch 3/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1519 - mse: 0.1519 - mae: 0.0608 - root_mean_squared_error: 0.3897 - mean_squared_logarithmic_error: 0.0354
Epoch 4/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1519 - mse: 0.1519 - mae: 0.0608 - root_mean_squared_error: 0.3897 - mean_squared_logarithmic_error: 0.0354
Epoch 5/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1519 - mse: 0.1519 - mae: 0.0608 - root_mean_squared_error: 0.3897 - mean_squared_logarithmic_error: 0.0354
Epoch 6/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1519

...

35/35 [==============================] - 1s 22ms/step - loss: 0.1095 - mse: 0.1095 - mae: 0.0821 - root_mean_squared_error: 0.3309 - mean_squared_logarithmic_error: 0.0276
Epoch 90/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1081 - mse: 0.1081 - mae: 0.0785 - root_mean_squared_error: 0.3288 - mean_squared_logarithmic_error: 0.0278
Epoch 91/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1076 - mse: 0.1076 - mae: 0.0814 - root_mean_squared_error: 0.3280 - mean_squared_logarithmic_error: 0.0275
Epoch 92/100
35/35 [==============================] - 1s 19ms/step - loss: 0.1083 - mse: 0.1083 - mae: 0.0870 - root_mean_squared_error: 0.3291 - mean_squared_logarithmic_error: 0.0278
Epoch 93/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1019 - mse: 0.1019 - mae: 0.0762 - root_mean_squared_error: 0.3192 - mean_squared_logarithmic_error: 0.0261
Epoch 94/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1000 - mse:

35/35 [==============================] - 1s 22ms/step - loss: 0.1075 - mse: 0.1075 - mae: 0.0804 - root_mean_squared_error: 0.3278 - mean_squared_logarithmic_error: 0.0274
Epoch 75/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1078 - mse: 0.1078 - mae: 0.0835 - root_mean_squared_error: 0.3283 - mean_squared_logarithmic_error: 0.0279
Epoch 76/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1071 - mse: 0.1071 - mae: 0.0869 - root_mean_squared_error: 0.3273 - mean_squared_logarithmic_error: 0.0277
Epoch 77/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1111 - mse: 0.1111 - mae: 0.0907 - root_mean_squared_error: 0.3333 - mean_squared_logarithmic_error: 0.0283
Epoch 78/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1119 - mse: 0.1119 - mae: 0.0910 - root_mean_squared_error: 0.3345 - mean_squared_logarithmic_error: 0.0287
Epoch 79/100
35/35 [==============================] - 1s 23ms/step - loss: 0.1004 - mse:

35/35 [==============================] - 1s 19ms/step - loss: 0.1040 - mse: 0.1040 - mae: 0.0787 - root_mean_squared_error: 0.3225 - mean_squared_logarithmic_error: 0.0270
Epoch 60/100
35/35 [==============================] - 1s 21ms/step - loss: 0.0956 - mse: 0.0956 - mae: 0.0736 - root_mean_squared_error: 0.3092 - mean_squared_logarithmic_error: 0.0258
Epoch 61/100
35/35 [==============================] - 1s 17ms/step - loss: 0.0974 - mse: 0.0974 - mae: 0.0704 - root_mean_squared_error: 0.3120 - mean_squared_logarithmic_error: 0.0258
Epoch 62/100
35/35 [==============================] - 1s 17ms/step - loss: 0.0998 - mse: 0.0998 - mae: 0.0821 - root_mean_squared_error: 0.3160 - mean_squared_logarithmic_error: 0.0272
Epoch 63/100
35/35 [==============================] - 1s 17ms/step - loss: 0.1074 - mse: 0.1074 - mae: 0.0775 - root_mean_squared_error: 0.3277 - mean_squared_logarithmic_error: 0.0288
Epoch 64/100
35/35 [==============================] - 1s 17ms/step - loss: 0.1000 - mse:

Epoch 1/100
35/35 [==============================] - 7s 33ms/step - loss: 0.1489 - mse: 0.1489 - mae: 0.0993 - root_mean_squared_error: 0.3859 - mean_squared_logarithmic_error: 0.0348
Epoch 2/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1461 - mse: 0.1461 - mae: 0.1108 - root_mean_squared_error: 0.3822 - mean_squared_logarithmic_error: 0.0342
Epoch 3/100
35/35 [==============================] - 1s 21ms/step - loss: 0.1436 - mse: 0.1436 - mae: 0.0959 - root_mean_squared_error: 0.3790 - mean_squared_logarithmic_error: 0.0331
Epoch 4/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1408 - mse: 0.1408 - mae: 0.0982 - root_mean_squared_error: 0.3752 - mean_squared_logarithmic_error: 0.0330
Epoch 5/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1391 - mse: 0.1391 - mae: 0.0934 - root_mean_squared_error: 0.3730 - mean_squared_logarithmic_error: 0.0330
Epoch 6/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1394

35/35 [==============================] - 1s 22ms/step - loss: 0.1092 - mse: 0.1092 - mae: 0.0919 - root_mean_squared_error: 0.3305 - mean_squared_logarithmic_error: 0.0279
Epoch 90/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1047 - mse: 0.1047 - mae: 0.0924 - root_mean_squared_error: 0.3235 - mean_squared_logarithmic_error: 0.0277
Epoch 91/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1036 - mse: 0.1036 - mae: 0.0788 - root_mean_squared_error: 0.3219 - mean_squared_logarithmic_error: 0.0270
Epoch 92/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1077 - mse: 0.1077 - mae: 0.0913 - root_mean_squared_error: 0.3282 - mean_squared_logarithmic_error: 0.0280
Epoch 93/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1022 - mse: 0.1022 - mae: 0.0877 - root_mean_squared_error: 0.3198 - mean_squared_logarithmic_error: 0.0265
Epoch 94/100
35/35 [==============================] - 1s 22ms/step - loss: 0.1218 - mse:

In [22]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/propv9/'

In [23]:
total_training_time

[90.75029706954956,
 88.22049474716187,
 88.92065453529358,
 90.08306121826172,
 91.54003143310547,
 89.73444318771362,
 90.17072606086731,
 90.05193758010864,
 87.1650059223175,
 89.27816891670227]

In [24]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 29.863827355702718


In [25]:
total_training_loss_history

[{'loss': [0.14860272407531738,
   0.14597737789154053,
   0.14287041127681732,
   0.14105752110481262,
   0.1406068056821823,
   0.1373162716627121,
   0.13860636949539185,
   0.13927200436592102,
   0.135357066988945,
   0.13445977866649628,
   0.1353757679462433,
   0.13166792690753937,
   0.13139504194259644,
   0.12971755862236023,
   0.128521129488945,
   0.12563608586788177,
   0.126736119389534,
   0.1255148947238922,
   0.126347154378891,
   0.12525403499603271,
   0.1284223198890686,
   0.12262287735939026,
   0.120177261531353,
   0.12258207052946091,
   0.12276908755302429,
   0.1193096712231636,
   0.12490914762020111,
   0.12038693577051163,
   0.11873718351125717,
   0.1166721060872078,
   0.11890502274036407,
   0.11925597488880157,
   0.11899825930595398,
   0.11743723601102829,
   0.11791170388460159,
   0.11599243432283401,
   0.1316135972738266,
   0.11835766583681107,
   0.11573132872581482,
   0.11678092926740646,
   0.1155601367354393,
   0.11398731172084808,
   

In [26]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(10):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 0.0406446340183417
avg mae of training: 0.026876706381638846


In [27]:
total_scores

[[0.15188556909561157,
  0.15188556909561157,
  0.10242875665426254,
  0.389724999666214,
  0.03817938268184662],
 [0.1416640281677246,
  0.1416640281677246,
  0.09532841295003891,
  0.37638282775878906,
  0.037461359053850174],
 [0.18427322804927826,
  0.18427322804927826,
  0.12911348044872284,
  0.42927056550979614,
  0.04636944457888603],
 [0.1445358395576477,
  0.1445358395576477,
  0.0669800266623497,
  0.38017868995666504,
  0.038528185337781906],
 [0.1445358395576477,
  0.1445358395576477,
  0.0669800266623497,
  0.38017868995666504,
  0.038528185337781906],
 [0.1445358395576477,
  0.1445358395576477,
  0.0669800266623497,
  0.38017868995666504,
  0.038528185337781906],
 [0.14921118319034576,
  0.14921118319034576,
  0.10027535259723663,
  0.3862786293029785,
  0.03494926169514656],
 [0.1528100073337555,
  0.1528100073337555,
  0.08562660217285156,
  0.39090919494628906,
  0.03632195293903351],
 [0.17811480164527893,
  0.17811480164527893,
  0.11386711895465851,
  0.42203649878

In [1]:
a = [[0.15188556909561157,
  0.15188556909561157,
  0.10242875665426254,
  0.389724999666214,
  0.03817938268184662],
 [0.1416640281677246,
  0.1416640281677246,
  0.09532841295003891,
  0.37638282775878906,
  0.037461359053850174],
 [0.18427322804927826,
  0.18427322804927826,
  0.12911348044872284,
  0.42927056550979614,
  0.04636944457888603],
 [0.1445358395576477,
  0.1445358395576477,
  0.0669800266623497,
  0.38017868995666504,
  0.038528185337781906],
 [0.1445358395576477,
  0.1445358395576477,
  0.0669800266623497,
  0.38017868995666504,
  0.038528185337781906],
 [0.1445358395576477,
  0.1445358395576477,
  0.0669800266623497,
  0.38017868995666504,
  0.038528185337781906],
 [0.14921118319034576,
  0.14921118319034576,
  0.10027535259723663,
  0.3862786293029785,
  0.03494926169514656],
 [0.1528100073337555,
  0.1528100073337555,
  0.08562660217285156,
  0.39090919494628906,
  0.03632195293903351],
 [0.17811480164527893,
  0.17811480164527893,
  0.11386711895465851,
  0.4220364987850189,
  0.04045278951525688],
 [0.15455670654773712,
  0.15455670654773712,
  0.10803474485874176,
  0.39313700795173645,
  0.036209262907505035]]

In [2]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])

In [5]:
sum(test_mae)/10

[0.10242875665426254,
 0.09532841295003891,
 0.12911348044872284,
 0.0669800266623497,
 0.0669800266623497,
 0.0669800266623497,
 0.10027535259723663,
 0.08562660217285156,
 0.11386711895465851,
 0.10803474485874176]

In [31]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(10):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 0.05153743475675583
avg mae: 0.031187151620785395
avg are: 0.013337250426411628


In [32]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.11686195060610771
median testing mse: 0.15054837614297867


In [33]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.0858258344233036
median testing mae: 0.09780188277363777


In [34]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.039952997118234634


In [35]:
test_ARE

[0.03877791,
 0.03936545,
 0.035840187,
 0.039952997,
 0.039952997,
 0.039952997,
 0.04171563,
 0.039952997,
 0.04171563,
 0.042890716]

In [6]:
a = [0.03877791,
 0.03936545,
 0.035840187,
 0.039952997,
 0.039952997,
 0.039952997,
 0.04171563,
 0.039952997,
 0.04171563,
 0.042890716]
sum(a)/10

0.04001175109999999